<table>
    <tr>
      <td>Minería de datos y Paradigma BigData (<b>MIN</b>) - Facultad de Informática - UCM
      </td>
      <td>
      <img src="https://biblioteca.ucm.es/data/cont/media/www/pag-88746//escudo.jpg"  width=50/>
      </td>
     </tr>
</table>




### Práctica 1 - Web scrapping de Ultimate Fighting Championship
Pablo C. Cañizares

No olvidéis los nombres de los dos componentes del grupo

### Se nos encarga preparar un dataset de luchadores de la compañía para futuras decisiones estratégicas
Entre estas decisiones radican posibles `matchmaking` de combates, herramientas para ojeadores, potenciación de carreras o incluso `¡despidos!`

Empezamos por acceder a la página de estadísicas de la reputada compañias de artes marciales mixtas UFC `ufcstats.com`. Específicamente, vamos a analizar inicialmente al vigente campeón del peso pluma el hispano-georgiano `Ilia "El matador" Topuria`. Para ello, accedemos al enlace `http://ufcstats.com/fighter-details/54f64b5e283b0ce7`

In [72]:
# (Solo ejecutar si hay problemas con la version de numpy)
#import sys
#!{sys.executable} -m pip uninstall -y pyarrow numexpr bottleneck streamlit


In [73]:
import pandas as pd
from bs4 import BeautifulSoup
import requests


def loadPage(urlIn):
    page = requests.get(urlIn)
    soup = BeautifulSoup(page.text, 'html.parser')  # le pasamos el texto en HTML para que lo analice
    return soup

url = "http://ufcstats.com/fighter-details/54f64b5e283b0ce7"
soup = loadPage(url) 
soup

<!DOCTYPE html>

<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html class="no-js ie8 lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>
    Stats | UFC
  </title>
<meta content="" name="description"/>
<meta content="" name="viewport"/>
<link href="/blocks/main.css?ver=534525" rel="stylesheet"/>
<script src="/js/vendor/modernizr-2.6.2.min.js"></script>
<script>
    (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
    (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
    m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
    })(window,document,'script','//www.google-analytics.com/analytics.js','ga');

    ga('create', 'UA-2855164-1', 'auto');
    ga('send

In [74]:
print(soup.head.title.text)


    Stats | UFC
  


**Ejercicio 1.** (1 punto) Extrae el nombre del luchador, así como su record y alias

In [75]:
#Nombre
def getName(soup):
    # ...
    nombre=soup.find("span",{"class":"b-content__title-highlight"}).text
    nombre=nombre.strip()
    return nombre
    
#Record
def getRecord(soup):
    # ...
    record=soup.find("span",{"class":"b-content__title-record"}).text
    record=record.strip()
    palabras=record.split(":")
    record=palabras[1].strip()
    return record

#NickName
def getNickName(soup):
    # ...
    nickname=soup.find("p",{"class":"b-content__Nickname"}).text
    nickname=nickname.strip()
    return nickname

nombre = getName(soup)
record = getRecord(soup)
nickname= getNickName(soup)

print(record)
print(nombre)
print(nickname)

17-0-0
Ilia Topuria
El Matador


In [76]:
#Hacemos un diccionario con las stats iniciales
def getInitialStats(soup):
    dict ={"NAME":"","RECORD":"","NICKNAME":""}
    dict["NAME"] = getName(soup)
    dict["RECORD"] = getRecord(soup)
    dict["NICKNAME"] = getNickName(soup)
    return dict

initialStatsDict = getInitialStats(soup)
print(initialStatsDict)

{'NAME': 'Ilia Topuria', 'RECORD': '17-0-0', 'NICKNAME': 'El Matador'}


**Ejercicio 2.** (2 puntos) Extrae las caracteristicas básicas del luchador: HEIGHT, WEIGHT, REACH, STANCE, DOB

In [77]:
def cleanBasicStat(cadena):
    # ...
    basicStat=[]
    for item in cadena :
        t=item.text.strip()
        par=t.split(":")
        t=par[1].strip()
        basicStat.append(t)
    return basicStat

def getBasicStatsList(soup):
    # ...
    div=soup.select_one("div.l-page__container>div>div")
    listado=div.find("ul")
    items=listado.find_all("li")
    cleanStats=cleanBasicStat(items)
    return cleanStats
    
cleanStats = getBasicStatsList(soup)
print(cleanStats)

['5\' 7"', '155 lbs.', '69"', 'Orthodox', 'Jan 21, 1997']


In [78]:
#Utiliza las funciones anteriores para extraer todos los datos
def getBasicStats(soup):
    #Obtenemos la lista de estadísticas básicas
    cleanStats = getBasicStatsList(soup)
    
    #Haz un diccionario con los valores
    basicStatsDict = {"HEIGHT":"", "WEIGHT":"", "REACH":"","STANCE":"","DOB":""}

    #Asignamos los valores (despues veremos un método más pro)
    basicStatsDict["HEIGHT"] = cleanStats[0]
    basicStatsDict["WEIGHT"] = cleanStats[1]
    basicStatsDict["REACH"] = cleanStats[2]
    basicStatsDict["STANCE"] = cleanStats[3]
    basicStatsDict["DOB"] = cleanStats[4]
    return basicStatsDict
    
basicStatsDict = getBasicStats(soup)
print(basicStatsDict)

{'HEIGHT': '5\' 7"', 'WEIGHT': '155 lbs.', 'REACH': '69"', 'STANCE': 'Orthodox', 'DOB': 'Jan 21, 1997'}


**Ejercicio 3.** (2 puntos) Extrae las caracteristicas de la carrera del luchador: 
- **SLpM** - Golpes significativos conectados por minuto.
- **Str. Acc.** - Precisión en golpes significativos.
- **SApM** - Golpes significativos absorbidos por minuto.
- **Str. Def.** - Defensa ante golpes significativos (porcentaje de golpes del oponente que no conectaron).
- **TD Avg.** - Derribos promedio logrados por cada 15 minutos.
- **TD Acc.** - Precisión en derribos.
- **TD Def.** - Defensa ante derribos (porcentaje de intentos de derribo del oponente que no tuvieron éxito).
- **Sub. Avg.** - Intentos de sumisión promedio por cada 15 minutos.

In [79]:
#Obten solo los resultados, en el siguiente apartado los adaptamos con diccionarios
def cleanCarrerStat(cadena):
    carrerStat=[]
    for item in cadena :
        t=item.text.strip()
        par=t.split(":")
        t=par[1].strip()
        carrerStat.append(t)
    return carrerStat

def getCareerStatsList(soup):
    cleanCareerStats =[]
    # ...
    div=soup.select_one("div.l-page__container>div>div+div>div>div")
    listado=div.find("ul")
    items=listado.find_all("li")
    carrStsIzq=cleanCarrerStat(items)
    div=soup.select_one("div.l-page__container>div>div+div>div>div+div")
    listado=div.find("ul")
    items=listado.find_all("li")
    items.pop(0)
    carrStsDer=cleanCarrerStat(items)
    cleanCareerStats=carrStsIzq+carrStsDer
    return cleanCareerStats

cleanCareerStats = getCareerStatsList(soup)
print(cleanCareerStats)

['4.81', '48%', '3.83', '64%', '1.96', '61%', '93%', '1.1']


In [80]:
def getCareerStats(soup):
    cleanCareerStats = getCareerStatsList(soup)
    #Hacemos otro diccionario
    dictCS = {"SLpM":"", "Str. Acc.":"", "SApM":"","Str. Def.":"","TD Avg.":"", "TD Acc.":"", "TD Def.":"", "Sub. Avg.":""}
    dictCS = dict(zip(dictCS.keys(), cleanCareerStats))
    return dictCS
    
careerStatsDict =  getCareerStats(soup)  
print(careerStatsDict)

{'SLpM': '4.81', 'Str. Acc.': '48%', 'SApM': '3.83', 'Str. Def.': '64%', 'TD Avg.': '1.96', 'TD Acc.': '61%', 'TD Def.': '93%', 'Sub. Avg.': '1.1'}


**Ejercicio 4** (1 punto). El siguiente paso es unificar las funciones anteriores. Crea una función *extractFighterStats* que obtenga una lista de las estadísticas extraidas de la página del luchador proporcionada.

Para ello, vamos a tener que es unificar el diccionario. Existen tres metodos distintos, vamos a utilizar el más directo y más elegante

In [81]:
#Asi se mezclan los diccionarios
fighterDict = initialStatsDict | basicStatsDict | careerStatsDict
print(fighterDict)

#Y así se obtienen las claves y valores. Puede que te sirva para despues:
print(fighterDict.keys())
print(fighterDict.values())

{'NAME': 'Ilia Topuria', 'RECORD': '17-0-0', 'NICKNAME': 'El Matador', 'HEIGHT': '5\' 7"', 'WEIGHT': '155 lbs.', 'REACH': '69"', 'STANCE': 'Orthodox', 'DOB': 'Jan 21, 1997', 'SLpM': '4.81', 'Str. Acc.': '48%', 'SApM': '3.83', 'Str. Def.': '64%', 'TD Avg.': '1.96', 'TD Acc.': '61%', 'TD Def.': '93%', 'Sub. Avg.': '1.1'}
dict_keys(['NAME', 'RECORD', 'NICKNAME', 'HEIGHT', 'WEIGHT', 'REACH', 'STANCE', 'DOB', 'SLpM', 'Str. Acc.', 'SApM', 'Str. Def.', 'TD Avg.', 'TD Acc.', 'TD Def.', 'Sub. Avg.'])
dict_values(['Ilia Topuria', '17-0-0', 'El Matador', '5\' 7"', '155 lbs.', '69"', 'Orthodox', 'Jan 21, 1997', '4.81', '48%', '3.83', '64%', '1.96', '61%', '93%', '1.1'])


In [82]:
import pandas as pd
from bs4 import BeautifulSoup
import requests


def loadPage(urlIn):
    page = requests.get(urlIn)
    soup = BeautifulSoup(page.text, 'html.parser')  # le pasamos el texto en HTML para que lo analice
    return soup

def extractFighterStats(urlIn):
    #Obtenemos la página con soup
    # ...
    soup = loadPage(urlIn) 

    
    #Adquirimos los diccionarios (llamando a las funciones ya hechas)
    dictInitial = getInitialStats(soup)
    dictBasic = getBasicStats(soup)
    dictCareer = getCareerStats(soup)

    #Hacemos un diccionario juntando los diccionarios anteriores
    dictFighter = dictInitial|dictBasic|dictCareer

    #Devolvemos una lista con los valores
    return list(dictFighter.values())

listFighter = extractFighterStats(url)
print(listFighter)

['Ilia Topuria', '17-0-0', 'El Matador', '5\' 7"', '155 lbs.', '69"', 'Orthodox', 'Jan 21, 1997', '4.81', '48%', '3.83', '64%', '1.96', '61%', '93%', '1.1']


**Ejercicio 5** (1 puntos). Ahora vamos a probar a extraer los datos de varios luchadores. Explora la página `http://ufcstats.com/statistics/fighters`, la cual lista distintos peleadores, extrae todos los enlaces y carga las stats de los nombres. 

In [83]:
urlFighters = "http://ufcstats.com/statistics/fighters"
page = requests.get(urlFighters)
soup = BeautifulSoup(page.text, 'html.parser')  # le pasamos el texto en HTML para que lo analice

In [84]:
def extractFighterLinksList(soup):
    # ...                    
    retList = []
    table = soup.find("table").select("tr")
    table.pop(0)
    table.pop(0)
    for tr in table:
        a=tr.find("a", href=True)
        retList.append(a['href'])
    return retList
                
linkList = extractFighterLinksList(soup)
print(linkList)

['http://ufcstats.com/fighter-details/93fe7332d16c6ad9', 'http://ufcstats.com/fighter-details/15df64c02b6b0fde', 'http://ufcstats.com/fighter-details/59a9d6dac61c2540', 'http://ufcstats.com/fighter-details/4961467134abd8be', 'http://ufcstats.com/fighter-details/b361180739bed4b0', 'http://ufcstats.com/fighter-details/3329d692aea4dc28', 'http://ufcstats.com/fighter-details/841695e02c99a521', 'http://ufcstats.com/fighter-details/2f5cbecbbe18bac4', 'http://ufcstats.com/fighter-details/c0ed7b208197e8de', 'http://ufcstats.com/fighter-details/5140122c3eecd307', 'http://ufcstats.com/fighter-details/c9f6385af6df66d7', 'http://ufcstats.com/fighter-details/aa6e591c2a2cdecd', 'http://ufcstats.com/fighter-details/7279654c7674cd24', 'http://ufcstats.com/fighter-details/f689bd7bbd14b392', 'http://ufcstats.com/fighter-details/1c5879330d42255f', 'http://ufcstats.com/fighter-details/989b85f6540c86b1', 'http://ufcstats.com/fighter-details/2620f3eb21c79614', 'http://ufcstats.com/fighter-details/83b00f7597

**Ejercicio 6 (1 punto)**. Es momento de hacer nuestro mini dataset de luchadores. Para ello, itera la lista de enlaces, y extrae las stats de todos ellos. Cada vez que extraigas stats de un peleador, pon tu programa a dormir por 1 segundo ( utiliza `time.sleep(1)`)

In [85]:
import time

def extractFighterListOfLists(linkList):
    lists=[]
    i=0
    for link in linkList:
        print("link: ", link)
        stats=extractFighterStats(link)
        lists.append(stats)
        print("Extracted {0} stat: ".format(i),stats)
        i = i+1
    return lists

listOfLists = extractFighterListOfLists(linkList)
print(listOfLists)

link:  http://ufcstats.com/fighter-details/93fe7332d16c6ad9
Extracted 0 stat:  ['Tom Aaron', '5-3-0', '', '--', '155 lbs.', '--', '', 'Jul 13, 1978', '0.00', '0%', '0.00', '0%', '0.00', '0%', '0%', '0.0']
link:  http://ufcstats.com/fighter-details/15df64c02b6b0fde
Extracted 1 stat:  ['Danny Abbadi', '4-6-0', 'The Assassin', '5\' 11"', '155 lbs.', '--', 'Orthodox', 'Jul 03, 1983', '3.29', '38%', '4.41', '57%', '0.00', '0%', '77%', '0.0']
link:  http://ufcstats.com/fighter-details/59a9d6dac61c2540
Extracted 2 stat:  ['Nariman Abbasov', '28-4-0', 'Bayraktar', '5\' 8"', '155 lbs.', '66"', 'Orthodox', 'Feb 01, 1994', '3.00', '20%', '5.67', '46%', '0.00', '0%', '66%', '0.0']
link:  http://ufcstats.com/fighter-details/4961467134abd8be
Extracted 3 stat:  ['Darion Abbey', '9-5-0', '', '6\' 2"', '265 lbs.', '80"', 'Orthodox', 'Feb 25, 1993', '8.44', '50%', '14.06', '28%', '0.00', '0%', '0%', '0.0']
link:  http://ufcstats.com/fighter-details/b361180739bed4b0
Extracted 4 stat:  ['David Abbott', '1

KeyboardInterrupt: 

**Ejercicio 7 (1 punto)**. Ahora te toca crear un dataset, y guardarlo con el nombre `fighters.csv`

In [ ]:
#Primero creamos el DF
df_fighters = pd.DataFrame(listOfLists)
df_fighters.columns = ['NAME', 'RECORD', 'NICKNAME', 'HEIGHT', 'WEIGHT', 'REACH', 'STANCE', 'DOB',  
    'SLpM', 'Str. Acc.', 'SApM', 'Str. Def.',  'TD Avg.', 'TD Acc.', 'TD Def.', 'Sub. Avg.']
print(df_fighters)

                   NAME         RECORD              NICKNAME  HEIGHT  \
0             Tom Aaron          5-3-0                            --   
1          Danny Abbadi          4-6-0          The Assassin  5' 11"   
2       Nariman Abbasov         28-4-0             Bayraktar   5' 8"   
3          Darion Abbey          9-5-0                         6' 2"   
4          David Abbott        10-15-0                  Tank   6' 0"   
5      Hamdy Abdelwahab   7-1-0 (1 NC)            The Hammer   6' 2"   
6    Mansur Abdul-Malik          9-0-1                         6' 2"   
7   Shamil Abdurakhimov         20-8-0                 Abrek   6' 3"   
8          Hiroyuki Abe  8-15-3 (1 NC)               Abe Ani   5' 6"   
9            Daichi Abe          6-2-0                        5' 11"   
10           Papy Abedi         10-4-0               Makambo  5' 11"   
11        Ricardo Abreu          5-1-0               Demente  5' 11"   
12        Klidson Abreu  15-4-0 (1 NC)            White Bear   6

In [ ]:
#Ahora lo guardamos
df_fighters.to_csv('fighters.csv', sep=',', index=False)
print("Archivo CSV 'df_fighters.csv' creado.")

Archivo CSV 'df_fighters.csv' creado.


**Ejercicio 8** (1 punto). Ahora, vamos a intentar crear un dataset con los luchadores más importantes. Para ello, accede a la página `http://ufcstats.com/statistics/fighters?char=a&page=all` y selecciona únicamente los campeones. Cabe destacar que los campeones son aquellos que tienen el icono de un cinturón <img src="http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/belt.png" class="b-list__icon"> en la última celda

In [ ]:
# ...

print(listOfLists)

['http://ufcstats.com/fighter-details/399afbabc02376b5']
link:  http://ufcstats.com/fighter-details/399afbabc02376b5
Extracted 0 stat:  ['Tom Aspinall', '15-3-0 (1 NC)', '', '6\' 5"', '255 lbs.', '78"', 'Orthodox', 'Apr 11, 1993', '7.63', '67%', '3.62', '56%', '2.62', '80%', '100%', '1.3']
[['Tom Aspinall', '15-3-0 (1 NC)', '', '6\' 5"', '255 lbs.', '78"', 'Orthodox', 'Apr 11, 1993', '7.63', '67%', '3.62', '56%', '2.62', '80%', '100%', '1.3']]


**Ejercicio extra 1** Si ya has llegado hasta aqui, tienes que intentarlo, en el ejercicio anterior únicamente has obtenido los campeones que empiezan por la letra a. Obtiene todos los campeones de la 'a' a la 'z', modificando el caracter XX por una letra ``http://ufcstats.com/statistics/fighters?char=XX&page=all``

In [ ]:
champions = []
# ...
    
print(champions)

In [ ]:
#Creamos el DF
df_champs = pd.DataFrame(champions, columns=['NAME', 'RECORD', 'NICKNAME', 'HEIGHT', 'WEIGHT', 'REACH', 'STANCE', 'DOB', 'SLpM', 'Str. Acc.', 'SApM', 'Str. Def.', 'TD Avg.', 'TD Acc.', 'TD Def.', 'Sub. Avg.'])
print(df_champs)
#Y lo guardamos
df_champs.to_csv('fighters_champs.csv', sep=',', index=False)
print("Archivo CSV 'df_champs.csv' creado.")

**Ejercicio extra 2** Extrae todos los luchadores, no solo campeones

In [ ]:
fighters = []
# ...
    
print(fighters)


#Creamos el DF
df_fighters = pd.DataFrame(fighters, columns=['NAME', 'RECORD', 'NICKNAME', 'HEIGHT', 'WEIGHT', 'REACH', 'STANCE', 'DOB', 'SLpM', 'Str. Acc.', 'SApM', 'Str. Def.', 'TD Avg.', 'TD Acc.', 'TD Def.', 'Sub. Avg.'])
print(df_fighters)
#Y lo guardamos
df_fighters.to_csv('all_fighters.csv', sep=',', index=False)
print("Archivo CSV 'all_fighters.csv' creado.")


**Ejercicio extra 3** Extrae todas las peleas del evento UFC 308 con sus detalles y haz un dataframe `ufc308.csv`

In [ ]:
# ...